In [1]:
import torch

In [2]:
import torch.autograd as autograd

In [3]:
import torch.nn as nn

In [50]:
import torch.nn.functional as F

In [4]:
import torch.optim as optim

In [5]:
torch.manual_seed(1)

In [6]:
lstm = nn.LSTM(3, 3)

In [7]:
print(lstm)

LSTM(3, 3)


In [8]:
inputs = [autograd.Variable(torch.randn((1, 3))) for _ in range(5)]

In [9]:
print(input)

<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7ffb841c6dd8>>


In [14]:
hidden = (autograd.Variable(torch.randn(1, 1, 3)),
         autograd.Variable(torch.randn(1, 1, 3)))

In [15]:
print(torch.randn(1, 1, 3))


(0 ,.,.) = 
  0.5258 -0.2969 -0.0681
[torch.FloatTensor of size 1x1x3]



In [16]:
print(torch.randn((1, 1, 3)))


(0 ,.,.) = 
 -0.2831 -0.4705 -1.7655
[torch.FloatTensor of size 1x1x3]



In [17]:
for i in inputs:
    out, hidden = lstm(i.view(1, 1, -1), hidden)

In [18]:
print(inputs)

[Variable containing:
-0.1186  0.4903  0.8349
[torch.FloatTensor of size 1x3]
, Variable containing:
 0.8894  0.4148  0.0507
[torch.FloatTensor of size 1x3]
, Variable containing:
-0.9644 -2.0111  0.5245
[torch.FloatTensor of size 1x3]
, Variable containing:
 2.1332 -0.0822  0.8388
[torch.FloatTensor of size 1x3]
, Variable containing:
-1.3233  0.0701  1.2200
[torch.FloatTensor of size 1x3]
]


In [19]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)

In [20]:
print(inputs)

Variable containing:
(0 ,.,.) = 
 -0.1186  0.4903  0.8349

(1 ,.,.) = 
  0.8894  0.4148  0.0507

(2 ,.,.) = 
 -0.9644 -2.0111  0.5245

(3 ,.,.) = 
  2.1332 -0.0822  0.8388

(4 ,.,.) = 
 -1.3233  0.0701  1.2200
[torch.FloatTensor of size 5x1x3]



In [21]:
hidden = (autograd.Variable(torch.randn(1, 1, 3)), autograd.Variable(torch.randn(1, 1, 3)))

In [22]:
out, hidden = lstm(inputs, hidden)

In [23]:
print(out)

Variable containing:
(0 ,.,.) = 
 -0.1472 -0.0460  0.4003

(1 ,.,.) = 
  0.0184  0.0418  0.3138

(2 ,.,.) = 
 -0.0970  0.0020  0.2638

(3 ,.,.) = 
  0.0124 -0.1045  0.2449

(4 ,.,.) = 
  0.0132  0.0259  0.3515
[torch.FloatTensor of size 5x1x3]



In [24]:
print(hidden)

(Variable containing:
(0 ,.,.) = 
  0.0132  0.0259  0.3515
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
  0.0256  0.0577  0.7966
[torch.FloatTensor of size 1x1x3]
)


In [25]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

In [26]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

In [27]:
print(training_data)

[(['The', 'dog', 'ate', 'the', 'apple'], ['DET', 'NN', 'V', 'DET', 'NN']), (['Everybody', 'read', 'that', 'book'], ['NN', 'V', 'DET', 'NN'])]


In [28]:
type(training_data)

list

In [29]:
len(training_data)

2

In [30]:
word_to_ix = {}

In [31]:
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            print("word_to_ix[word]:{}, word: {}".format(word_to_ix[word], word))

word_to_ix[word]:0, word: The
word_to_ix[word]:1, word: dog
word_to_ix[word]:2, word: ate
word_to_ix[word]:3, word: the
word_to_ix[word]:4, word: apple
word_to_ix[word]:5, word: Everybody
word_to_ix[word]:6, word: read
word_to_ix[word]:7, word: that
word_to_ix[word]:8, word: book


In [32]:
print(word_to_ix)

{'The': 0, 'read': 6, 'apple': 4, 'Everybody': 5, 'that': 7, 'the': 3, 'book': 8, 'ate': 2, 'dog': 1}


In [33]:
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

In [34]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

In [35]:
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
               autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space)
        return tag_scores

In [36]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

In [37]:
loss_function = nn.NLLLoss()

In [38]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [45]:
inputs = prepare_sequence(training_data[0][0], word_to_ix)

In [46]:
print(training_data[0][1])

['DET', 'NN', 'V', 'DET', 'NN']


In [47]:
print(word_to_ix)

{'The': 0, 'read': 6, 'apple': 4, 'Everybody': 5, 'that': 7, 'the': 3, 'book': 8, 'ate': 2, 'dog': 1}


In [48]:
print(inputs)

Variable containing:
 0
 1
 2
 3
 4
[torch.LongTensor of size 5]



In [51]:
tag_scores = model(inputs)

In [52]:
print(tag_scores)

Variable containing:
-0.9565 -1.0443 -1.3325
-0.9508 -1.0677 -1.3101
-0.9737 -0.9903 -1.3830
-1.0014 -0.9567 -1.3925
-0.9760 -1.0320 -1.3209
[torch.FloatTensor of size 5x3]



In [53]:
for epoch in range(300):
    for sentence, tags in training_data:
        model.zero_grad()
        
        model.hidden = model.init_hidden()
        
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        tag_scores = model(sentence_in)
        
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

In [54]:
inputs = prepare_sequence(training_data[0][0], word_to_ix)

In [55]:
tag_scores = model(inputs)

In [56]:
print(tag_scores)

Variable containing:
-0.3421 -3.2324 -1.3853
-5.2787 -0.0462 -3.2173
-2.5584 -2.6201 -0.1628
-0.0436 -4.9419 -3.3363
-5.0088 -0.0287 -3.8361
[torch.FloatTensor of size 5x3]

